In [ ]:
import os
from pathlib import Path
from collections import defaultdict

BASE = Path('/content/drive/MyDrive/Research/CASTING ARTICLE/casting_512x512')
for item in sorted(BASE.iterdir()):
    if item.is_dir():
        print(f'  📁 {item.name}/')
    else:
        print(f'  📄 {item.name}')
ext_counts = defaultdict(int)
folder_counts = defaultdict(lambda: defaultdict(int))

for root, dirs, files in os.walk(BASE):
    dirs.sort()
    level = Path(root).relative_to(BASE)
    indent = '  ' * len(level.parts)
    folder_name = Path(root).name

    # Count files by extension
    file_exts = defaultdict(int)
    for f in files:
        ext = Path(f).suffix.lower()
        file_exts[ext] += 1
        ext_counts[ext] += 1

    if files:
        ext_summary = ', '.join([f'{v} {k}' for k,v in file_exts.items()])
        print(f'  {indent}📁 {folder_name}/ → [{ext_summary}]')
    else:
        print(f'  {indent}📁 {folder_name}/ → [empty]')
for ext, count in sorted(ext_counts.items()):
    print(f'  {ext or "no extension"}: {count} files')
print('\n[4] LABEL FORMAT CHECK:')
txt_files = list(BASE.rglob('*.txt'))
yaml_files = list(BASE.rglob('*.yaml')) + list(BASE.rglob('*.yml'))
print(f'  .txt label files found: {len(txt_files)}')
print(f'  .yaml config files found: {len(yaml_files)}')

if txt_files:
    sample_label = txt_files[0]
    print(f'\n  Sample label file: {sample_label.relative_to(BASE)}')
    print(f'  Contents: {sample_label.read_text().strip()[:200]}')

if yaml_files:
    for y in yaml_files:
        print(f'\n  YAML file: {y.relative_to(BASE)}')
        print(f'  Contents:\n{y.read_text().strip()}')
all_image_exts = {'.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff'}
class_counts = defaultdict(int)
all_images = []

for root, dirs, files in os.walk(BASE):
    folder = Path(root).name.lower()
    for f in files:
        if Path(f).suffix.lower() in all_image_exts:
            all_images.append(Path(root) / f)
            class_counts[folder] += 1

print(f'  Total images found: {len(all_images)}')
for cls, count in sorted(class_counts.items()):
    pct = count / len(all_images) * 100 if all_images else 0
    print(f'  Folder "{cls}": {count} images ({pct:.1f}%)')
print('\n[6] SAMPLE IMAGE PATHS (first 3):')
for img in all_images[:3]:
    print(f'  {img.relative_to(BASE)}')
print('\n[7] SPLIT DETECTION:')
split_keywords = ['train', 'val', 'test', 'valid']
found_splits = []
for kw in split_keywords:
    matches = [p for p in BASE.rglob('*') if kw in p.name.lower() and p.is_dir()]
    if matches:
        found_splits.append(kw)
        for m in matches:
            print(f'  ✅ Found "{kw}" folder: {m.relative_to(BASE)}')
if not found_splits:
    print('  ⚠️  No train/val split folders detected — raw class folders only')

print('\n' + '=' * 60)
print('ASSESSMENT COMPLETE — Share this output')
print('=' * 60)

In [ ]:
!pip install ultralytics -q

import os, shutil, random
import numpy as np
from pathlib import Path

BASE     = Path('/content/drive/MyDrive/Research/CASTING ARTICLE/casting_512x512')
WORK_DIR = Path('/content/casting_work')  # fast local SSD
SEED     = 42
random.seed(SEED)
np.random.seed(SEED)
CLASS_MAP = {'def_front': 0, 'ok_front': 1}

all_images = []  # list of (Path, class_int)
for folder, cls_id in CLASS_MAP.items():
    imgs = sorted((BASE / folder).glob('*.jpeg'))
    all_images.extend([(p, cls_id) for p in imgs])

random.shuffle(all_images)  # shuffle with fixed seed

print(f'Total images collected : {len(all_images)}')
print(f'  def_front (cls 0)    : {sum(1 for _,c in all_images if c==0)}')
print(f'  ok_front  (cls 1)    : {sum(1 for _,c in all_images if c==1)}')

# ── Stratified split: 1170 train / 130 val ───────────────────
def stratified_split(data, val_count=130, seed=42):
    random.seed(seed)
    class0 = [x for x in data if x[1] == 0]
    class1 = [x for x in data if x[1] == 1]

    # Proportional val split
    ratio = len(class0) / len(data)
    val0  = int(round(val_count * ratio))
    val1  = val_count - val0

    random.shuffle(class0)
    random.shuffle(class1)

    val_set   = class0[:val0]   + class1[:val1]
    train_set = class0[val0:]   + class1[val1:]

    print(f'\nStratified split:')
    print(f'  Train : {len(train_set)} images  '
          f'(def:{sum(1 for _,c in train_set if c==0)}, '
          f'ok:{sum(1 for _,c in train_set if c==1)})')
    print(f'  Val   : {len(val_set)} images  '
          f'(def:{sum(1 for _,c in val_set if c==0)}, '
          f'ok:{sum(1 for _,c in val_set if c==1)})')
    return train_set, val_set

TRAIN_ALL, VAL_SET = stratified_split(all_images, val_count=130)

LABEL_LINE = lambda cls_id: f'{cls_id} 0.5000 0.5000 1.0000 1.0000\n'

VAL_IMG_DIR = WORK_DIR / 'val/images'
VAL_LBL_DIR = WORK_DIR / 'val/labels'
VAL_IMG_DIR.mkdir(parents=True, exist_ok=True)
VAL_LBL_DIR.mkdir(parents=True, exist_ok=True)

for img_path, cls_id in VAL_SET:
    shutil.copy(img_path, VAL_IMG_DIR / img_path.name)
    (VAL_LBL_DIR / (img_path.stem + '.txt')).write_text(
        LABEL_LINE(cls_id))



In [ ]:
import yaml
import time
from ultralytics import YOLO

TRAINING_SIZES = [200, 400, 600, 900, 1170]
RESULTS = {}  # store metrics per size

for N in TRAINING_SIZES:
    print(f'\n{"="*55}')
    print(f'  TRAINING WITH {N} IMAGES  ({N/1170*100:.0f}% of full set)')
    print(f'{"="*55}')
    t_start = time.time()

    RUN_DIR     = WORK_DIR / f'run_{N}'
    TRAIN_IMG   = RUN_DIR / 'train/images'
    TRAIN_LBL   = RUN_DIR / 'train/labels'
    TRAIN_IMG.mkdir(parents=True, exist_ok=True)
    TRAIN_LBL.mkdir(parents=True, exist_ok=True)

    random.seed(SEED)  # reset seed for reproducibility
    cls0_pool = [x for x in TRAIN_ALL if x[1] == 0]
    cls1_pool = [x for x in TRAIN_ALL if x[1] == 1]
    random.shuffle(cls0_pool)
    random.shuffle(cls1_pool)

    ratio     = len(cls0_pool) / len(TRAIN_ALL)
    n_cls0    = int(round(N * ratio))
    n_cls1    = N - n_cls0

    selected  = cls0_pool[:n_cls0] + cls1_pool[:n_cls1]
    random.shuffle(selected)

    print(f'  Selected: {n_cls0} defective + {n_cls1} ok = {len(selected)} total')

    for img_path, cls_id in selected:
        shutil.copy(img_path, TRAIN_IMG / img_path.name)
        (TRAIN_LBL / (img_path.stem + '.txt')).write_text(
            LABEL_LINE(cls_id))

    data_cfg = {
        'path' : str(RUN_DIR),
        'train': 'train/images',
        'val'  : str(WORK_DIR / 'val/images'),  # fixed val
        'nc'   : 2,
        'names': {0: 'defective', 1: 'non_defective'}
    }
    yaml_path = RUN_DIR / 'data.yaml'
    with open(yaml_path, 'w') as f:
        yaml.dump(data_cfg, f, default_flow_style=False)

    model = YOLO('yolov8s.pt')
    model.train(
        data         = str(yaml_path),
        epochs       = 80,
        imgsz        = 640,
        batch        = 16,
        optimizer    = 'Adam',
        lr0          = 0.01,
        lrf          = 0.1,          # final lr = lr0 * lrf = 0.001
        weight_decay = 0.0005,
        seed         = SEED,
        hsv_h        = 0.0,          # no hue shift (grayscale images)
        hsv_s        = 0.0,          # no saturation shift
        hsv_v        = 0.4,          # brightness variation
        fliplr       = 0.5,
        degrees      = 15.0,
        scale        = 0.2,
        mosaic       = 0.3,
        mixup        = 0.2,
        label_smoothing = 0.1,
        verbose      = False,
        project      = str(WORK_DIR / 'runs'),
        name         = f'yolov8s_n{N}',
        exist_ok     = True
    )

      val_cfg = {
        'path' : str(WORK_DIR),
        'train': 'val/images',       # placeholder
        'val'  : 'val/images',
        'nc'   : 2,
        'names': {0: 'defective', 1: 'non_defective'}
    }
    val_yaml = WORK_DIR / 'val_only.yaml'
    with open(val_yaml, 'w') as f:
        yaml.dump(val_cfg, f, default_flow_style=False)

    metrics = model.val(data=str(val_yaml), verbose=False)

    P   = float(metrics.box.p.mean())
    R   = float(metrics.box.r.mean())
    M50 = float(metrics.box.map50)
    M95 = float(metrics.box.map)
    F1  = 2*P*R/(P+R) if (P+R) > 0 else 0

    RESULTS[N] = {
        'precision' : round(P,   4),
        'recall'    : round(R,   4),
        'mAP50'     : round(M50, 4),
        'mAP5095'   : round(M95, 4),
        'F1'        : round(F1,  4)
    }

    elapsed = (time.time() - t_start) / 60
    print(f'\n  ✅ n={N} done in {elapsed:.1f} min')
    print(f'     Precision : {P:.4f}')
    print(f'     Recall    : {R:.4f}')
    print(f'     mAP@0.5   : {M50:.4f}')
    print(f'     F1-Score  : {F1:.4f}')

    shutil.rmtree(TRAIN_IMG)
    shutil.rmtree(TRAIN_LBL)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

SAVE_PATH = Path('/content/drive/MyDrive/Research/'
                 'CASTING ARTICLE/learning_curve_yolov8s.png')

sizes = list(RESULTS.keys())
map50 = [RESULTS[n]['mAP50'] for n in sizes]
f1    = [RESULTS[n]['F1']    for n in sizes]
prec  = [RESULTS[n]['precision'] for n in sizes]
rec   = [RESULTS[n]['recall']    for n in sizes]

sat_idx = None
for i in range(1, len(map50)):
    if (map50[i] - map50[i-1]) < 0.005:
        sat_idx = i
        break
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
fig.suptitle(
    'YOLOv8s Learning Curve: Performance vs. Training Set Size\n'
    '(Fixed 130-image validation set, Random seed = 42)',
    fontsize=12, fontweight='bold', y=1.02
)

ax = axes[0]
ax.plot(sizes, map50, 'o-', color='#1565C0', linewidth=2.5,
        markersize=8, label='mAP@0.5', zorder=5)
ax.plot(sizes, f1,    's--', color='#C62828', linewidth=2.5,
        markersize=8, label='F1-Score', zorder=5)

for i, n in enumerate(sizes):
    ax.annotate(f'{map50[i]:.3f}', (n, map50[i]),
                textcoords='offset points', xytext=(0, 10),
                ha='center', fontsize=9, color='#1565C0',
                fontweight='bold')
    ax.annotate(f'{f1[i]:.3f}', (n, f1[i]),
                textcoords='offset points', xytext=(0, -16),
                ha='center', fontsize=9, color='#C62828',
                fontweight='bold')

if sat_idx:
    sat_x = sizes[sat_idx]
    ax.axvspan(sat_x, sizes[-1], alpha=0.10, color='#2E7D32',
               label=f'Saturation zone (>{sat_x} images)')
    ax.axvline(x=sat_x, color='#2E7D32', linestyle=':',
               linewidth=2, alpha=0.8)
    ax.text(sat_x + 35, min(map50) + 0.005,
            f'Saturation\n>{sat_x} images',
            fontsize=8, color='#2E7D32', alpha=0.9)

ax.set_xlabel('Training Set Size (images)', fontsize=8)
ax.set_ylabel('Performance Score', fontsize=11)
ax.set_title('(a) Detection Performance', fontsize=8)
ax.set_xticks(sizes)
ax.set_xticklabels([str(n) for n in sizes])
ax.set_ylim([min(min(map50), min(f1)) - 0.02, 1.005])
ax.legend(loc='lower right', fontsize=9)
ax.grid(True, alpha=0.3, linestyle='--')

# Panel (b) — Precision and Recall separately
ax2 = axes[1]
ax2.plot(sizes, prec, '^-',  color='#6A1B9A', linewidth=2.5,
         markersize=8, label='Precision', zorder=5)
ax2.plot(sizes, rec,  'D--', color='#E65100', linewidth=2.5,
         markersize=8, label='Recall',    zorder=5)

for i, n in enumerate(sizes):
    ax2.annotate(f'{prec[i]:.3f}', (n, prec[i]),
                 textcoords='offset points', xytext=(0, 10),
                 ha='center', fontsize=9, color='#6A1B9A',
                 fontweight='bold')
    ax2.annotate(f'{rec[i]:.3f}', (n, rec[i]),
                 textcoords='offset points', xytext=(0, -16),
                 ha='center', fontsize=9, color='#E65100',
                 fontweight='bold')

if sat_idx:
    ax2.axvspan(sat_x, sizes[-1], alpha=0.10, color='#2E7D32')
    ax2.axvline(x=sat_x, color='#2E7D32', linestyle=':',
                linewidth=2, alpha=0.8)

ax2.set_xlabel('Training Set Size (images)', fontsize=11)
ax2.set_ylabel('Performance Score', fontsize=11)
ax2.set_title('(b) Precision–Recall Breakdown', fontsize=8)
ax2.set_xticks(sizes)
ax2.set_xticklabels([str(n) for n in sizes])
ax2.set_ylim([min(min(prec), min(rec)) - 0.02, 1.005])
ax2.legend(loc='lower right', fontsize=9)
ax2.grid(False, alpha=0.3, linestyle='--')

plt.tight_layout()
plt.savefig(str(SAVE_PATH), dpi=300, bbox_inches='tight',
            facecolor='white')
plt.savefig('/content/learning_curve_yolov8s.png',
            dpi=300, bbox_inches='tight', facecolor='white')


In [ ]:
import csv
from pathlib import Path

all_subset = TRAIN_ALL + VAL_SET

rows = []
for img_path, cls_id in all_subset:
    split = 'val' if (img_path, cls_id) in VAL_SET else 'train'
    rows.append({
        'filename'  : img_path.name,
        'folder'    : img_path.parent.name,
        'class_id'  : cls_id,
        'class_name': 'defective' if cls_id == 0 else 'non_defective',
        'split'     : split
    })

# Sort for reproducibility
rows.sort(key=lambda x: (x['split'], x['folder'], x['filename']))

save_path = '/content/drive/MyDrive/Research/CASTING ARTICLE/image_subset_list.csv'
with open(save_path, 'w', newline='') as f:
    writer = csv.DictWriter(f,
             fieldnames=['filename','folder','class_id',
                        'class_name','split'])
    writer.writeheader()
    writer.writerows(rows)


In [ ]:
# Generate data.yaml
import yaml

cfg = {
    'dataset'     : 'Casting Product Image Data for Quality Inspection',
    'source'      : 'https://www.kaggle.com/datasets/ravirajsinh45/'
                    'real-life-industrial-dataset-of-casting-product',
    'subset'      : '1300 images (see image_subset_list.csv)',
    'train_images': 1170,
    'val_images'  : 130,
    'seed'        : 42,
    'nc'          : 2,
    'names'       : {0: 'defective', 1: 'non_defective'},
    'image_size'  : 640,
    'note'        : 'Bounding boxes are whole-part annotations. '
                    'Class 0=defective (def_front), '
                    'Class 1=non_defective (ok_front).'
}

yaml_path = '/content/drive/MyDrive/Research/CASTING ARTICLE/data.yaml'
with open(yaml_path, 'w') as f:
    yaml.dump(cfg, f, default_flow_style=False, sort_keys=False)
